In [ ]:
import json
import ast
import re
import getpass

from usgs_datatools import doi

In [ ]:
username = 'vgarbulet@contractor.usgs.gov'
password = getpass.getpass('USGS AD Password: ')
DoiSession = doi.DoiSession(env='staging')
DoiSession.doi_authenticate(username, password)
print ("Successfully authenticated.")

In [ ]:
with open('Report.txt', 'r', errors='replace') as infile:
    line = infile.readline()
    count = 0
    while line != '':
        count = count + 1
        if line.strip():
            pair = line.split(" : ")
            in_doi = pair[0]
            ipds = pair[1]
            
            # Get the doi
            print(str(count) + ". " + in_doi + "     " + ipds)
            out_doi = DoiSession.get_doi(in_doi)
            
            # Check if data release ipds exists and if ends with \n char
            new_line = 0
            for pubIpds in out_doi['ipdsNumbers']:
                if pubIpds['ipdsNumber'][-1] == '\n':
                    new_line = 1
                    invalid_ipds = pubIpds['ipdsNumber'].rstrip()
            
                    # Removes duplicates
                    duplicates = 0
                    for list_item in out_doi['ipdsNumbers']:

                        if list_item['ipdsNumber'] == invalid_ipds:
                            dupliactes = 1
                            out_doi['ipdsNumbers'].remove(list_item)
                            break
                            
                    if duplicates:
                        print('DUPLICATE IPDS\n')
                    else:
                        pubIpds['ipdsNumber'] = invalid_ipds

            
            # update DOI if data release ipds exists and ends with \n 
            if new_line == 1:
                update_request = DoiSession.doi_update(out_doi)

                # If there were any errors, save the doi and error message to errors file
                if 'error' in update_request.keys():
                    print("FAILED\n")
                    with open('invalid.txt', 'a') as error_file:
                        error_file.write(str(in_doi) + " : " + str(ipds))
                        error_file.write("\n" + str(update_request['message']) + "\n\n")
                else:
                    print('SUCCESS\n')
            else:
                print('SKIPPING\n')
                
        line = infile.readline()

In [ ]:
with open('Report.txt', 'r', errors='replace') as infile:
    line = infile.readline()
    count = 1
    while line != '':
        if line.strip():
            pair = line.split(" : ")
            in_doi = pair[0]
            ipds = pair[1].rstrip()
            
            # Get the doi
            print(str(count) + ". " + in_doi + "      " + ipds)
            out_doi = DoiSession.get_doi(in_doi)
            
            # Check if publication ipds exists
            pub_ipds = 0
            data_ipds = 0
            out_doi['noDataReleaseAvailableReason'] = ""
            if 'ipdsNumbers' in out_doi.keys():
                pubIpds_list = out_doi['ipdsNumbers']
                for pubIpds in pubIpds_list:
                    if pubIpds['ipdsType'] == 'PUBLICATION':
                        pub_ipds = 1
                        
                    if pubIpds['ipdsType'] == 'DATA_RELEASE':
                        data_ipds = 1
                
                # Add the new IPDS# to the record if it doesn't exist
                if data_ipds == 0:
                    out_doi['ipdsNumbers'].append({'ipdsNumber': ipds, 'ipdsType':'DATA_RELEASE'})
                else:
                    line = infile.readline()
                    count = count + 1
                    continue
                    
            # Check authors for invalid position
            if len(out_doi['authors']) == 1:
                if out_doi['authors'][0]['position'] == -1:
                    out_doi['authors'][0]['position'] = 0                    
            
            # Check if date or dateType is set
            if 'date' in out_doi.keys() and not out_doi['date']:
                out_doi['dateType'] = ""
                
            # Check for invalid characters
            if 'description' in out_doi.keys() and out_doi['description']:
                out_doi['description'] = out_doi['description'].encode("ascii", "ignore").decode('utf-8')

            if 'title' in out_doi.keys() and out_doi['title']:
                out_doi['title'] = out_doi['title'].encode("ascii", "ignore").decode('utf-8')

            if 'subject' in out_doi.keys() and out_doi['subject']:
                out_doi['subject'] = out_doi['subject'].encode("ascii", "ignore").decode('utf-8')
            
            # Check if publication crossref exists
            crossref = 0
            if 'relatedIdentifiers' in out_doi.keys():
                relatedIds = out_doi['relatedIdentifiers']
                for relatedId in relatedIds:
                    if relatedId['usgsRelationSubType'] == 'PUBLICATION':
                        crossref = 1
            
            # Set noPublicationIdAvailable flag to True if there is no publication crossref or ipds
            if crossref == 0 and pub_ipds == 0:
                out_doi['noPublicationIdAvailable'] = True
            
            # Send update request to DOi Tool
            update_request = DoiSession.doi_update(out_doi)
            
            # If there was any errors, save the doi and error message to errors file
            if 'error' in update_request.keys() and \
               'DataCite' not in update_request['message']:
                print("FAILED\n")
                with open('errors.txt', 'a') as error_file:
                    error_file.write(str(in_doi) + " : " + str(ipds))
                    error_file.write("\n" + str(update_request['message']) + "\n\n")
                with open('failures.txt', 'a') as failures_file:
                    failures_file.write(str(in_doi) + " : " + str(ipds) + "\n")
            else:
                print("SUCCESS\n")
        line = infile.readline()
        count = count + 1